In [38]:
import pymolpro

p = pymolpro.Project("orbital")
# p.write_input("symmetry,nosym;geometry={o;h,o,r;h,o,r,h,theta};r=1 angstrom, theta=104 degree; rhf;locali,pipek,thrpip=1e-10;put,xml")
p.write_input("""
!symmetry,nosym
gprint,orbital
noorient;theta=20,r=1.4 b;
geometry={
2

H,0,0,0
H,r*cos(theta),r*sin(theta),0
}; {rhf;wf,2,1,0}
!locali,pipek,thrpip=1e-10
put,xml
""")
p.run(wait=True)

In [39]:
orbital_nodes = p.xpath("//orbitals/orbital[@occupation > 0.0]")

In [40]:
orbitals = [pymolpro.Orbital(node) for node in orbital_nodes]

In [41]:
orbitals = p.orbitals(minocc=1.0)

In [42]:
orbital_nodes[0].text.split()

['0.576919368270112',
 '0.121231522277131E-01',
 '0.203521242294181E-01',
 '0.7407567430851E-02',
 '0.0',
 '0.576919368270112',
 '0.1212315222771E-01',
 '-0.20352124229418E-01',
 '-0.74075674308508E-02',
 '0.0']

In [43]:
for orbital in orbitals:
    points, weights = orbital.grid(2, 'Gauss-Hermite', True, scale=1.0)
    print(points)
    # print(weights)
    values = p.evaluateOrbitals(points, ID=orbital.ID)['values']
    norm = 0
    centroid = [0, 0, 0]
    smzz = 0
    smxx = 0
    # print("weights",weights)
    # print("points",points)
    # print("values",values)
    for i in range(len(weights)):
        norm += pow(values[i], 2) * weights[i]
        smxx += pow(values[i], 2) * weights[i] * pow(points[i,0], 2)
        smzz += pow(values[i], 2) * weights[i] * pow(points[i,2], 2)
        for j in range(3):
            centroid[j] += pow(values[i], 2) * weights[i] * points[i,j]
    print(orbital.ID,"norm",norm,"centroid",centroid,orbital.centroid,"XX",smxx,"ZZ",smzz,orbital.attribute('moments'))

[[ 2.30873874  1.94665214 -1.0396195 ]
 [ 3.01988036 -0.0071934  -1.0396195 ]
 [ 2.30873874  1.94665214  1.0396195 ]
 [ 3.01988036 -0.0071934   1.0396195 ]
 [-0.53381399  0.91204756 -1.0396195 ]
 [ 0.17732764 -1.04179798 -1.0396195 ]
 [-0.53381399  0.91204756  1.0396195 ]
 [ 0.17732764 -1.04179798  1.0396195 ]]
1.1 norm 0.9061126526985805 centroid [1.1263280977690657, 0.40994990160562017, 0.0] [1.24303318623202, 0.452427079993506, 0.0] XX 3.3449951364020807 ZZ 0.9793344440505021 1.24303318623202 0.452427079993506 0.0 3.69158859709925 1.42667002695721 1.08080870643827 0.95024616840496 0.0 0.0 0.37644250968112


In [44]:
points


array([[ 2.30873874,  1.94665214, -1.0396195 ],
       [ 3.01988036, -0.0071934 , -1.0396195 ],
       [ 2.30873874,  1.94665214,  1.0396195 ],
       [ 3.01988036, -0.0071934 ,  1.0396195 ],
       [-0.53381399,  0.91204756, -1.0396195 ],
       [ 0.17732764, -1.04179798, -1.0396195 ],
       [-0.53381399,  0.91204756,  1.0396195 ],
       [ 0.17732764, -1.04179798,  1.0396195 ]])

In [45]:
orbitals[0].second_moment_eigenvalues

array([1.08080871, 1.08080871, 2.28762815])

In [46]:
orbitals[0].second_moment_eigenvectors

array([[ 0.34202014,  0.        , -0.93969262],
       [-0.93969262,  0.        , -0.34202014],
       [ 0.        ,  1.        , -0.        ]])

In [47]:
orbitals[0].local_second_moments

array([[2.1464571 , 0.38786429, 0.        ],
       [0.38786429, 1.22197976, 0.        ],
       [0.        , 0.        , 1.08080871]])

In [48]:
orbitals[0].centroid

[1.24303318623202, 0.452427079993506, 0.0]

In [49]:
import numpy as np

np.matmul(orbitals[0].second_moment_eigenvectors, orbitals[0].second_moment_eigenvectors.transpose())

array([[1.00000000e+00, 1.61124041e-17, 0.00000000e+00],
       [1.61124041e-17, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [50]:
for orbital in orbitals:
    print(orbital.ID, orbital.centroid, orbital.second_moment_eigenvalues)

1.1 [1.24303318623202, 0.452427079993506, 0.0] [1.08080871 1.08080871 2.28762815]


In [51]:
print(p.out)


 Working directory              : /var/folders/qd/f26v1rbd3g5341rsg3s9j4v80000gn/T//molpro.7IvMo10IWx/
 Global scratch directory       : /var/folders/qd/f26v1rbd3g5341rsg3s9j4v80000gn/T//molpro.7IvMo10IWx/
 Wavefunction directory         : /Users/peterk/wfu/
 Main file repository           : /var/folders/qd/f26v1rbd3g5341rsg3s9j4v80000gn/T//molpro.7IvMo10IWx/

 id        : peterk

 Nodes          nprocs
 pjk2022.connect    1
 GA implementation: MPI file
 GA implementation (serial work in mppx): MPI file

 Using customized tuning parameters: mindgm=1; mindgv=20; mindgc=4; mindgr=1; noblas=0; minvec=7
 default implementation of scratch files=df  


 Variables initialized (1023), CPU time= 0.00 sec
 
                                                                                 !symmetry,nosym
 gprint,orbital
 noorient;theta=20,r=1.4 b;
 geometry={
 2
 H,0,0,0
 H,r*cos(theta),r*sin(theta),0
 }; {rhf;wf,2,1,0}
                                                                             

In [52]:
len(points)

8

In [53]:
len(points[0])

3

In [54]:
print(p.out)



 Working directory              : /var/folders/qd/f26v1rbd3g5341rsg3s9j4v80000gn/T//molpro.7IvMo10IWx/
 Global scratch directory       : /var/folders/qd/f26v1rbd3g5341rsg3s9j4v80000gn/T//molpro.7IvMo10IWx/
 Wavefunction directory         : /Users/peterk/wfu/
 Main file repository           : /var/folders/qd/f26v1rbd3g5341rsg3s9j4v80000gn/T//molpro.7IvMo10IWx/

 id        : peterk

 Nodes          nprocs
 pjk2022.connect    1
 GA implementation: MPI file
 GA implementation (serial work in mppx): MPI file

 Using customized tuning parameters: mindgm=1; mindgv=20; mindgc=4; mindgr=1; noblas=0; minvec=7
 default implementation of scratch files=df  


 Variables initialized (1023), CPU time= 0.00 sec
 
                                                                                 !symmetry,nosym
 gprint,orbital
 noorient;theta=20,r=1.4 b;
 geometry={
 2
 H,0,0,0
 H,r*cos(theta),r*sin(theta),0
 }; {rhf;wf,2,1,0}
                                                                             

In [55]:
print(p.xml)


<?xml version="1.0"?>
<molpro xmlns="http://www.molpro.net/schema/molpro-output"
  xmlns:xsd="http://www.w3.org/1999/XMLSchema"
  xmlns:cml="http://www.xml-cml.org/schema"
  xmlns:stm="http://www.xml-cml.org/schema"
  xmlns:xhtml="http://www.w3.org/1999/xhtml">
 <job>
  <stm:metadataList>
   <stm:metadata name="dc:date" content="2022-12-06T16:56:48+00:00"/>
   <stm:metadata name="dc:creator" content="peterk"/>
   <stm:metadata name="cmlm:insilico" content="Molpro"/>
  </stm:metadataList>
  <platform pid="26727">
   <version major="2022" minor="3" SHA="037d71a8fcef44d454924c6e8a6f9a4fbf01cb74"
     integer_bits="64" parallelism="serial">
    2022.3
    <date year="2022" month="12" day="6" hour="16" minute="56" second="48">
     2022-12-06T16:56:48
    </date>
   </version>
   <licence id="peterk"/>
   <parallel processes="1" nodes="1" all_processes="1" openmp="1"/>
   <dimensions natom="400" nvalence="500" nvalence_pno="1000" nbasis="12000" nstate="100"
     nsymm="16" nrec="512" nprim=

In [56]:
points = orbitals[0].grid(3)


In [57]:
points

array([[ 1.75130682,  1.16507722, -0.49583225],
       [ 1.84077705,  0.91925979, -0.49583225],
       [ 1.92089144,  0.69914731, -0.49583225],
       [ 1.75130682,  1.16507722, -0.23423881],
       [ 1.84077705,  0.91925979, -0.23423881],
       [ 1.92089144,  0.69914731, -0.23423881],
       [ 1.75130682,  1.16507722,  0.        ],
       [ 1.84077705,  0.91925979,  0.        ],
       [ 1.92089144,  0.69914731,  0.        ],
       [ 1.39367927,  1.03491143, -0.49583225],
       [ 1.48314949,  0.789094  , -0.49583225],
       [ 1.56326388,  0.56898152, -0.49583225],
       [ 1.39367927,  1.03491143, -0.23423881],
       [ 1.48314949,  0.789094  , -0.23423881],
       [ 1.56326388,  0.56898152, -0.23423881],
       [ 1.39367927,  1.03491143,  0.        ],
       [ 1.48314949,  0.789094  ,  0.        ],
       [ 1.56326388,  0.56898152,  0.        ],
       [ 1.07344857,  0.91835699, -0.49583225],
       [ 1.1629188 ,  0.67253956, -0.49583225],
       [ 1.24303319,  0.45242708, -0.495

In [58]:
import scipy
x, w = scipy.special.roots_hermite(10)

In [59]:
x

array([-3.43615912, -2.53273167, -1.75668365, -1.03661083, -0.34290133,
        0.34290133,  1.03661083,  1.75668365,  2.53273167,  3.43615912])

In [60]:
w

array([7.64043286e-06, 1.34364575e-03, 3.38743945e-02, 2.40138611e-01,
       6.10862634e-01, 6.10862634e-01, 2.40138611e-01, 3.38743945e-02,
       1.34364575e-03, 7.64043286e-06])

In [61]:
import scipy
x, w = scipy.special.roots_hermite(100)
integral=0
import math
for i in range(len(w)):
    integral += math.exp(-abs(x[i]))*w[i]*math.exp(x[i]*x[i])
integral

1.995912100640584

In [62]:
import scipy
x, w = scipy.special.roots_laguerre(20)
integral=0
import math
for i in range(len(w)):
    integral += math.exp(-math.sqrt(4*x[i]*x[i]+1))*w[i]*math.exp(x[i])
integral

0.30096829971189465

In [63]:
import numgrid

In [64]:
# numgrid.get_angular_grid(5)

In [65]:
a=np.array([[1,2,3],[4,5,6]])

In [66]:
a.shape

(2, 3)

In [67]:
b=a[1,:]

In [68]:
b.shape

(3,)

In [69]:
c=np.array([1,2,3])

In [70]:
c.shape

(3,)

In [71]:
c[0]

1

In [72]:
type(c)

numpy.ndarray

In [73]:
l=[1 ,2, 3]

In [74]:
type(l)

list